In [1]:
include("../../src/JuliaMBD.jl")

Main.JuliaMBD

In [2]:
using Plots
using DifferentialEquations
using .JuliaMBD
using .JuliaMBD.Diagram

In [3]:
@model m1 begin
    @block begin
        con = Constant(value=1)#, out=Outport(:constantout))
        goto = Goto(tag=:a, in=InPort(:gotoin), out=OutPort(:gotoout))
        from = From(tag=:a, in=InPort(:fromin), out=OutPort(:fromout))
        gain = Gain(K=1, in=InPort(:gainin), out=OutPort(:gainout))
        from2 = From(tag=:x, in=InPort(:fromin1), out=OutPort(:fromout1))
        gain2 = Gain(K=1, in=InPort(:gainin2), out=OutPort(:gainout2))
    end
    @connect begin
        con.out => goto.in
        from.out => gain.in
        from2.out => gain2.in
    end
end

m1 (generic function with 1 method)

In [4]:
b = m1()
JuliaMBD.connecttag(b.blocks)
for x = JuliaMBD.tsort(JuliaMBD.allcomponents(b))
    println(JuliaMBD._expr(x))
end

ErrorException: Did not find Goto tag x

In [6]:
b = Goto(tag=:a)

"Block(Goto)"

In [10]:
JuliaMBD._expr(b.outports[1].outs[1])

:(())

In [5]:
JuliaMBD.gotoports

Dict{Symbol, Vector{Main.JuliaMBD.AbstractPortBlock}} with 1 entry:
  :a => []

In [3]:
## RLC

@model RLC begin
    @parameter begin
        R
        L
        C
    end
    @xmlmodel("RLC.drawio")
end

RLC (generic function with 1 method)

In [4]:
## RLC test

@model Test begin
    @parameter begin
        R
        L
        C
        voltage
    end

    @block begin
        system = RLC(R=R, L=L, C=C)
        source = Step(steptime=0.1, finalvalue=voltage)
    end

    @connect begin
        source.out => system.in
    end

    @scope begin
        source.out => v
        system.out => i
    end
end

Test (generic function with 1 method)

In [5]:
m = @compile Test(R=10, L=100e-3, C=10e-6, voltage=5)
result = JuliaMBD.simulate(m, tspan=(0,1))
plot(result)


┌ Warning: No goto tag jumpprefix_y
└ @ Main.JuliaMBD /workspaces/JuliaMBD.jl/src/_compile.jl:33
┌ Warning: No goto tag jumpprefix_y
└ @ Main.JuliaMBD /workspaces/JuliaMBD.jl/src/_compile.jl:33


┌ Warning: No goto tag jumpprefix_y
└ @ Main.JuliaMBD /workspaces/JuliaMBD.jl/src/_compile.jl:33


UndefVarError: UndefVarError: jumpprefix_y not defined

In [3]:
@model m2 begin
    @block begin
        con = Ramp(slope=1)
        tri = TrigonometricFunction()
    end
    @connect begin
        con.out => tri.in
    end
end

m2 (generic function with 1 method)

In [4]:
b = m2()

"Block(m2)"

In [5]:
JuliaMBD.expr_sfunc(b)

:(function m2_sfunc(; time = 0)
      var"##318" = 0.0
      var"##317" = 1
      begin
          var"##320" = time
      end
      var"##316" = var"##320"
      var"##319" = 0.0
      begin
          #= /workspaces/JuliaMBD.jl/src/predefined/Sources/Ramp.jl:24 =#
          var"##315" = if var"##316" < var"##318"
                  #= /workspaces/JuliaMBD.jl/src/predefined/Sources/Ramp.jl:25 =#
                  var"##319"
              else
                  #= /workspaces/JuliaMBD.jl/src/predefined/Sources/Ramp.jl:27 =#
                  var"##317" * (var"##316" - var"##318") + var"##319"
              end
      end
      begin
          var"##323" = var"##315"
      end
      var"##321" = var"##323"
      var"##322" = sin(var"##321")
      begin
      end
      ()
  end)